# STL file display

This notebook shows how to display STL files used for 3D printing in a notebook.

Displaying relies on the *numpy-stl* and the *ipyvolume* libraries.

Upload a STL **ascii** file (*binary files not supported*)

In [1]:
# imports
from io import StringIO

from stl import mesh
from traitlets import dlink
import ipyvolume as ipv
import ipywidgets as widgets
from ipyupload import FileUpload

class STL_Displayer:
    
    def __init__(self):
        # widgets creation
        self._file_upload = FileUpload(accept='*.stl')
        self._file_name = widgets.Text()
        self._wireframe_flag = widgets.Checkbox(
                    value=False,
                    description='Wireframe',
                    disabled=False)
        self._color_picker = widgets.ColorPicker(
                    concise=False,
                    description='Pick a color',
                    value='white',
                    disabled=False)
        self.ui = widgets.VBox([])
        self._update_ui()

        # links between widgets
        dlink((self._file_upload, 'value'), (self._file_name, 'value'), self._get_filename)
        self._color_picker.observe(self._show_stl, 'value')
        self._wireframe_flag.observe(self._show_stl, 'value')
        self._file_upload.observe(self._show_stl, 'value')
    
    # display STL file
    def _show_stl(self, *args):
        # bridge: FileUpload - stl.mesh
        content = next(iter(self._file_upload.value.values()))['content']
        io = StringIO()
        try:
            io.write(content.decode('utf-8'))
            io.seek(0)
            name, data = mesh.Mesh.load(io)
            m = mesh.Mesh(data, True, name=name, speedups=True)
        except:
            self._file_name.value = 'Provided STL file not supported'
            m = None
            
        mymesh = None

        if m is not None:
            ipv.figure()
            ipv.style.use('minimal')
            ipv.plot_mesh(m.x, m.z, m.y, color=self._color_picker.value, wireframe=self._wireframe_flag.value)
            mymesh = ipv.gcc()
            
        self._update_ui(mymesh)

    def _update_ui(self, mymesh=None):
        if mymesh is None:
            self.ui.children = (widgets.HBox([self._file_upload, self._file_name]), self._wireframe_flag, self._color_picker)
        else:
            self.ui.children = (widgets.HBox([self._file_upload, self._file_name]), self._wireframe_flag, self._color_picker, mymesh)

    # get filename from FileUpload dict
    def _get_filename(self, file_upload):
        if len(file_upload.values()) == 0:
            return ''
        return next(iter(file_upload.values()))['metadata']['name']

In [2]:
s = STL_Displayer()
s.ui